# Fortran formulae to Python String 

### Purpose 
Transform output strings from Mathematica to Fortran-compatible (or FortranParser-module compatible). 

### Usage 
Define inputs below and run all cells 


In [1]:
import re 

In [2]:
re.__version__

'2.2.1'

# User definable variables 

**Input**: Set absolute paths of Mathematica output files and then where to put Fortran-ready files

In [34]:
mathematicaOutputPaths = [
    "/home/degnaiyu/Työpöytä/kanditutkielma/final_codes/python_converter/Formulae_MathematicaOutput/probabilityFunctionFortranForm", 
    "/home/degnaiyu/Työpöytä/kanditutkielma/final_codes/python_converter/Formulae_MathematicaOutput/potentialEnergyLocalEnergyFortranForm", 
    "/home/degnaiyu/Työpöytä/kanditutkielma/final_codes/python_converter/Formulae_MathematicaOutput/kineticEnergy3LocalEnergyFortranForm", 
    "/home/degnaiyu/Työpöytä/kanditutkielma/final_codes/python_converter/Formulae_MathematicaOutput/kineticEnergy2LocalEnergyFortranForm", 
    "/home/degnaiyu/Työpöytä/kanditutkielma/final_codes/python_converter/Formulae_MathematicaOutput/kineticEnergy1LocalEnergyFortranForm"
    ]

In [35]:

whereToPut_paths = [
    "/home/degnaiyu/Työpöytä/kanditutkielma/final_codes/python_converter/Formulae_PythonCompatible/probabilityFunctionFortranForm", 
    "/home/degnaiyu/Työpöytä/kanditutkielma/final_codes/python_converter/Formulae_PythonCompatible/potentialEnergyLocalEnergyFortranForm", 
    "/home/degnaiyu/Työpöytä/kanditutkielma/final_codes/python_converter/Formulae_PythonCompatible/kineticEnergy3LocalEnergyFortranForm", 
    "/home/degnaiyu/Työpöytä/kanditutkielma/final_codes/python_converter/Formulae_PythonCompatible/kineticEnergy2LocalEnergyFortranForm", 
    "/home/degnaiyu/Työpöytä/kanditutkielma/final_codes/python_converter/Formulae_PythonCompatible/kineticEnergy1LocalEnergyFortranForm"
    ]


## Checklist for formula validity 

**Input (optional)** : We expect that the following do not appear in the output of Mathematica: 

In [36]:
checkList = ['Arg']

## Fortran to python dictionary 

**Input (optional)**: Not complete! Only supposed to apply to this project. 

In [37]:
mapping = {
    'E**': 'exp', 
    'Sqrt': 'sqrt', 
    'Abs': 'abs', 
    '\n': '' , 
    "\\" : '', 
    '**':'^', 
}

# Functions 

In [38]:
def readFileToString(absolutePath: str) -> str:  
    '''
    Read file by absolute path. Returns string 
    
    
    Parameters
    ----------
        absolutePath: absolute path of the file 
        
    
    Return
    ------
        Returns the content of the file as string 
    
    '''
    
    file = open(absolutePath, 'r') 
    return file.read()





In [39]:
def checkListFunction(string, checkList: list ) -> list: 
    '''
    Check whether there are unwanted elements in string following the checklist of unwanted elements 
    
    
    Parameters
    ----------
        string: string which we are checking 
        
        
    Return
    ------
        list of unwanted elements in the string
    
    '''
    unwantedList = []
    
    for unwanted in checkList: 
        if unwanted in string: 
            unwantedList.append(unwanted)
            
            
    return unwantedList
            

    
    
    
def printingForUnwantedElements(isLocalEnergy: bool, listOfUnwantedElements: list) -> None: 
    '''
    Printing for unwanted elements in the Mathematica output 
    '''
    
    if isLocalEnergy: 
        prompt = f'Local energy has {len(listOfUnwantedElements)} unwanted elements:'
        print(prompt)
        print(len(prompt)*'-')
        print(listOfUnwantedElements)
    else: 
        prompt = f'Probability function has {len(listOfUnwantedElements)} unwanted elements:'
        print(prompt)
        print(len(prompt)*'-')
        print(listOfUnwantedElements)
        
                

In [40]:
def fortranFormToPythonForm(string: str, mappingDict: dict) -> str: 
    '''
    Transform fortran formula to python formula using the above defined mapping 
    
    
    Parameters
    ---------
        string: input string containing fortran formulae generated by Mathematica 
        
        mappingDict: mapping dictionary, rules for transforming  fortran formulae to python formulae 
        
    Return
    ------
        string which is the input transformed into python formula
    
    '''
    
    transcript = string
    
    # transforming 
    for fortranForm, pythonForm in mappingDict.items(): 
        transcript = transcript.replace(fortranForm, pythonForm)
        
    return transcript
    

In [41]:
def savePythonFormToFile(string: str, absolutePath: str, mode = 'w') -> None:
    '''
    Save string to a file. Can overwrite or append 
    
    Parameters
    ----------
         string: string that is to be written to a file 
         
         absolutePath: filename or file absolute path 
         
         mode: writing mode. Whether to add  or overwrite content to a file. 
         
     Return
     ------
         The file should be created 
    '''
    df = open(absolutePath, mode ) 
    df.write(string )
    df.close()

In [42]:
def findVariables(exprStr: str) -> tuple:
    '''
    Extract variables in expression string. 
    
    Parameters
    ---------
        exprStr: expression as string. Supposed to be Python-compatible formula (at least Fortran, but not tested)
        
    
    Return
    ------
    Tuple of two lists. 
    
        First one: alphanumerically-ordered coordinate list e.g.
        ['x1', 'x2', 'x3', 'y1', 'y2', 'y3', 'z1', 'z2', 'z3']
        
        Second one: alphanumerically-ordered  parameters list 
        e.g. ['A1', 'A2']
        
    '''
    
    patternForCoordinates = r'[xyz]{1}[123]{1}'                   # pattern for: x1, x2, y1 etc. 
    patternForParameters = r'[A-Z]{1}\d{1}'                # pattern for: A1, A2, ... Z9  
    
    coordinateList =sorted(list(set( re.findall(patternForCoordinates, exprStr)  )))
    parametersList =sorted(list(set(  re.findall(patternForParameters, exprStr) )))
    
    return coordinateList, parametersList 
    
    

In [43]:
def regroupToCoordinateTriple_findVariables(coordinateList: list, isCoordinateTripletInTuple: bool = False ) -> list: 
    '''
    Regrouping the coordinates from the output of the previous function `findVariables` into form [(x1, y1, z1), (x2, y2, z2)]
    
    Parameters
    ----------
        coordinateList: output coordinates from findVariables-function 
        
        isCoordinateTripletInTuple: whether the output coordinate triples be separated by parentheses (see Return)
        
    Return
    ------
        list of traditional coordinatetriplets [(x1, y1, z1), (x2, y2, z2)] or in ordered list without parentheses [x1, y1, z1, x2, y2, z2 ]
        
    Warning
    -------
        User check that the coordinateList is the output of the function `findVariables` 
    '''
    
    tripletList = []   # save all triplets 
    
    
    # each particle's components 
    xCoordinates = []
    yCoordinates = []
    zCoordinates = []
    
    
    # separating each particle's component into their lists 
    for component in coordinateList: 
        if 'x' in component: 
            xCoordinates.append(component) 
        elif 'y' in component: 
            yCoordinates.append(component)
        elif 'z' in component:   
            zCoordinates.append(component)
        else: 
            raise Exception("No other than x,y,z components")
    
    
    
    
    if isCoordinateTripletInTuple: 
        # forming triplets 
        for x,y,z in zip(xCoordinates, yCoordinates, zCoordinates): 
            tripletList.append((x,y,z))
    else: 
        # forming ordered list 
        for x,y,z in zip(xCoordinates, yCoordinates, zCoordinates): 
            tripletList.append(x)
            tripletList.append(y)
            tripletList.append(z)
        
        
    return tripletList
    
    
def removeSpaces(expr:str): 
    return expr.replace(' ', '')
    
    

In [44]:
# show variables in the function expressions, to be used in Fortran program variable definition 
def showVariables(functionString: str) -> tuple : 

    # find coordinates and parameters
    coordinates, parameterStringList  = findVariables(functionString)

    # regrouping coordinates 
    coordinates = regroupToCoordinateTriple_findVariables(coordinates)


    # for functions usage, define list of ordered variables and parameters
    mapping =  coordinates + parameterStringList  # global variable: used in probability and local energy functions definition

    return mapping, len(mapping) 


In [49]:

def main(): 

    functionStrings = [readFileToString(x) for x in mathematicaOutputPaths]   # read mathematica output files to string 


    functionStrings_transformed = [fortranFormToPythonForm(x, mappingDict=mapping) for x in functionStrings]      # transform to Fortran-compatible expressions 

    functionStrings_transformed = [removeSpaces(x) for x in functionStrings_transformed] # remove spaces

    # save Fortran-ready strings into files 
    if len(functionStrings_transformed) ==len(whereToPut_paths): 
        for functionString, path in zip(functionStrings_transformed, whereToPut_paths): 
            savePythonFormToFile(functionString, absolutePath=path )

        print("SAVING SUCCESSFUL")
        print("Here are the variables in the expressions")
        print(showVariables(functionStrings_transformed[0]))
    else: 
        raise Exception("The length of the output path not same as number of function expressions!")





# Executing the functions 

Comment out if you have fortran forms from Mathematica: 

In [50]:
main()

SAVING SUCCESSFUL
Here are the variables in the expressions
(['x1', 'y1', 'z1', 'x2', 'y2', 'z2', 'x3', 'y3', 'z3', 'A1', 'A2', 'A3'], 12)


# Small testing of eval built-in function 

In [ ]:

# if __name__ == "__main__":
    

#     def probabilityfunction(R: list , parametersList: list ):  
#         x1, y1, z1 = R[0]
#         x2, y2, z2 = R[1]
#         x3, y3, z3 = R[2]

#         A1, A2 = parametersList



#         return eval(probability_pythonTransformed)
    
#     import math 
#     value = probabilityfunction(R = [(1,1,1), (2,2,2), (3,3,3)],
#                         parametersList= (1,1) )
#     print(value )